#INSTALL AND IMPORT DEPENDENCIES

In [1]:
# !pip install git+https://github.com/GabrieleSgroi/genetic_algorithm
# !sudo apt update
# !sudo apt-get install -y xvfb ffmpeg
# !pip install gym[all]
# !pip install ufal.pybox2d
# !pip install gymnasium
# !pip install pygame
# !pip install  pyvirtualdisplay
# !pip install box2d-py

import os
import numpy as np
import genetic_alg
from genetic_alg.ga import GeneticAlg
from genetic_alg.mutations import gaussian_mutation
from genetic_alg.crossovers import single_point_crossover
from genetic_alg.selections import rank_selection, boltzmann_selection
from genetic_alg.utils import flatten_weights, reshape_weights
import IPython
import pyvirtualdisplay
import base64
import imageio
import gymnasium as gym
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_agg import FigureCanvas
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Lambda
# Set up a virtual display for rendering OpenAI gym environments.
# display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()
env = gym.make("LunarLanderContinuous-v3", render_mode='rgb_array')
print(env.action_space)
print(env.observation_space)

Box(-1.0, 1.0, (2,), float32)
Box([ -2.5        -2.5       -10.        -10.         -6.2831855 -10.
  -0.         -0.       ], [ 2.5        2.5       10.        10.         6.2831855 10.
  1.         1.       ], (8,), float32)


We use the continuous version of the lunar lander environment present in OpenAI Gym.

The action consists of a two-dimensional vector with values bounded between -1 and 1. The first entry controls the main engine: it varies the power from 0 (-1) to 100% (+1). The main engine cannot work below 50% power. The second entry controls the lateral engines: in the interval [-1, -0,5] fires the left engine, in the interval [-0.5, 0.5] turn the side engines off, [0.5, 1] fires the right engine.

The observed state is a 8-dimensional vector consisting of:
- x coordinate of the lander;
- y coordinate of the lander;
- horizontal velocity;
- vertical velocity;
- angle of the lander;
- angular velocity of the lander around its center;
- boolean indicating contact of left leg with the ground;
- boolean indicating contact of right leg with the ground.

The landing pad is located at the origin.

In [2]:
import numpy as np
from numpy import load
# save np.load
# np_load_old = np.load

np.load.__defaults__=(None, True, True, 'ASCII')

In [3]:
def load_ga(dir, fitness_fn, mutation_fn, crossover_fn,
                 selection_fn, keep_parents=0, callback=None):
    '''Returns the loaded GeneticAlg instance'''
    loaded=GeneticAlg([], fitness_fn, mutation_fn, crossover_fn,
                 selection_fn, keep_parents=0, callback=callback)
    loaded.population=load(dir+'/saved_model/population.npy' ).tolist()
    loaded.fitness=load(dir+'/saved_model/fitness.npy' ).tolist()
    loaded.num_generations=load(dir+'/saved_model/generation.npy')
    loaded.fitness_hist=load(dir+'/saved_model/hist.npy').tolist()
    loaded.best_fitness=load(dir+'/saved_model/best_fitness.npy')
    loaded.best_solution=load(dir+'/saved_model/best_sol.npy')
    
    return loaded

In [6]:
!pip install imageio==2.19.3
!pip install imageio-ffmpeg==0.4.7


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached imageio_ffmpeg-0.4.7-py3-none-win_amd64.whl.metadata (1.6 kB)
Using cached imageio_ffmpeg-0.4.7-py3-none-win_amd64.whl (22.6 MB)


#EVALUATION UTILITIES

In [4]:
def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

def create_policy_eval_video(policy, env, filename, num_episodes=3, fps=30):
  '''Create a video of the given policy'''
  filename = filename + ".mp4"
  with imageio.get_writer(filename, fps=fps) as video:
    for _ in range(num_episodes):
      terminated = False
      truncated = False
#       print(env.reset())
      obs= np.asarray(env.reset()[0])
#       print(obs)
      obs=np.expand_dims(obs, axis=0)
      video.append_data(env.render())
      reward=0
      frame_counter=0
      while not terminated and not truncated:
        action=policy.predict(obs)[0]
        obs, rew, terminated, truncated, info = env.step(action)
#         print(obs, rew, terminated, truncated, info)
        reward+=rew
        obs=np.expand_dims(np.array(obs), axis=0)
        video.append_data(env.render())
        frame_counter+=1
      print('Episode reward:', reward, 'Number of frames:', frame_counter)
  return embed_mp4(filename)

def evaluate_policy(policy, env, num_episodes=100):
     '''Evaluate the policy. Returns two lists containing episode rewards and
        episode number of frames'''
     rews=[]
     frames=[]
     for i in range(num_episodes):
        reward=0
        frame_counter=0
        terminated = False
        truncated = False
        print(env.reset())
        obs= np.asarray(env.reset()[0])
        obs=np.expand_dims(obs, axis=0)
        while not terminated and not truncated:
            action=policy.predict(obs)[0]
            obs, rew, terminated, truncated, info = env.step(action)
            reward+=rew
            obs=np.expand_dims(np.array(obs), axis=0)
            frame_counter+=1
        rews.append(reward)
        frames.append(frame_counter)
     return rews, frames

Let's visualize the performance of an agent acting randomly.

In [5]:
class RandomModel():
    '''Creates a model that takes random action'''
    def __init__(self, action_dim):
        self.dim=action_dim

    def predict(self, data):
        rand=np.random.uniform(range(self.dim))
        return [rand]

rand_model=RandomModel(2)
create_policy_eval_video(rand_model, env, 'random', num_episodes=5)

RecursionError: maximum recursion depth exceeded

#Genetic algorithm functions

Here we define the functions needed by the genetic algorithm.

1.   Mutation function: we will use gaussian mutation with a mutation probability of 0.1. This means that each variable of the newly created child has 0.1 probability to be added to a value sampled from a gaussian distribution with mean 0 and standard deviation 0.1.
2.   Crossover function: We use single point crossover with 0.5 probability. This means that, selected 2 parents, the created child has a probability of 0.5 of being obtained by single point crossover from the two parents and a 0.5 probability of being equal to parent 1. Single point crossover selects a random index $i$ from 0 to the length of the chromosomes and creates a new chromosome made up by the variables of the chromosome of parent 1 up to position $i$ and by the variables of the chromosome of parent 2 from position $i$ to the end.
3.Fitness function: the fitness assigned to each solution is the total reward obtained in an episode. Notice that this does not make use of the time shape of the reward provided by the environment.
4. Selection function: each solution has a probability of being selected as a parent equal to $\frac{e^{f_i/T}}{\sum e^{f_j/T}}$, where $f_i$ is the fitness of the solution $i$ and the sum in the denominator runs over all the solution in the population. The temperature $T$ of the Boltzmann selection starts at 1 and it is decayed exponentially with the number of generations $n$ as $T=e^{-n/50}$ up to a minimum of 0.1.

In [12]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [13]:
def mutation_fn(sol):
    return gaussian_mutation(sol,mutation_rate=0.1, std=0.1)

def crossover_fn(sol):
    return single_point_crossover(sol, crossover_p=0.5)

def fit_func(sol):
    fitness=0
    model.set_weights(reshape_weights(model,sol))
    for i in range(1):
        obs=env.reset()[0]
        terminated = False
        truncated = False
        while not terminated and not truncated:
            obs=np.expand_dims(obs, axis=0)
            action=model.predict(obs, verbose=0)[0]
            obs, rew, terminated, truncated, info= env.step(action)
            fitness+=rew
    return fitness

def selection_fn(fitness):
    #Normalize fitnesses before passing to exponential, this will avoid scale dependence
    fitness=(fitness-np.mean(fitness))/(np.std(fitness))
    return boltzmann_selection(fitness, T=1.)

def generate_initial_pop(pop_size):
    models=[]
    initial_pop=[]
    for i in range(pop_size):
        inp=Input((8,))
        x=inp
        for i in range(2):
            x=Dense(256, activation='tanh')(x)
        out=Dense(2, activation='tanh')(x)
        models.append(Model(inputs=inp, outputs=out, trainable=False))
        initial_pop.append(flatten_weights(models[-1]))
    return initial_pop, models[-1]


def T_scheduler(inst):
    n_gen=inst.num_generations
    T=max(3*2**(-n_gen/50), 0.3)
    def selection_fn(fitness):
    #Normalize fitnesses before passing to exponential, this will avoid scale dependence
        fitness=(fitness-np.mean(fitness))/(np.std(fitness))
        return boltzmann_selection(fitness, T=T)
    inst.selection_fn=selection_fn
    print('Temperature for Boltzmann selection', T)

def save_call(inst):
    inst.save(save_path) #save and log every generation

# Training Loop

Training loop with automatic resuming previous run.

In [18]:
!rm -r /kaggle/working/saved_model

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [26]:
generations_run=1

def save_video_call(inst):
    n_gen=inst.num_generations
    if n_gen % 50==0:
        if not os.path.exists(save_path+'/videos'):
            os.makedirs(save_path+'/videos')
        idx=np.argmax(inst.fitness)
        current_best=inst.population[idx]
        model.set_weights(reshape_weights(model,current_best))
        create_policy_eval_video(model, env, save_path+'/videos/'+str(n_gen), num_episodes=5)
        print('')

save_path=''

if os.path.exists(save_path+'/saved_model'):
    unused, model=generate_initial_pop(1)
    ga=load_ga(save_path, fit_func, mutation_fn, crossover_fn, None, keep_parents=5, callback=[save_video_call,T_scheduler, save_call])
    n_gen=ga.num_generations
    T=max(3*2**(-n_gen/50), 0.3)    #Set selection function with the right temperature
    def selection_fn(fitness):
        fitness=(fitness-np.mean(fitness))/(np.std(fitness))
        return boltzmann_selection(fitness, T=T)
    ga.selection_fn=selection_fn
    print('Num parameters:', len(ga.population[0]))
    ga.run(generations_run)

else:
    initial_pop, model=generate_initial_pop(1)
    print('Num parameters:', len(initial_pop[0]))
    ga=GeneticAlg(initial_pop, fit_func, mutation_fn, crossover_fn, selection_fn, keep_parents=5, callback=[save_video_call, T_scheduler, save_call])
    ga.run(generations_run)

ga.save(save_path)

Num parameters: 68610


C:\Users\HP\AppData\Local\Temp\ipykernel_18344\2670713613.py:22: RuntimeWarning: invalid value encountered in divide
  fitness=(fitness-np.mean(fitness))/(np.std(fitness))


ValueError: probabilities contain NaN

#Visualization of the results

Let's visualize the trend of the mean fitness and standard deviation (shaded area) of the population over the generations.

In [13]:
save_path='/kaggle/working/'
initial_pop, model=generate_initial_pop(1)
ga=load_ga(save_path, fit_func, mutation_fn, crossover_fn, selection_fn, keep_parents=5)

fit_mean=[]
fit_std=[]
for el in ga.fitness_hist:
    fit_mean.append(el[2])
    fit_std.append(el[3])

fit_mean=np.array(fit_mean)
fit_std=np.array(fit_std)
plt.figure(figsize=(10,10))
plt.plot(fit_mean)
plt.fill_between(list(range(len(fit_mean))), fit_mean-fit_std, fit_mean+fit_std, alpha=0.4)
plt.savefig(save_path+'/pop_fitness.png')
plt.ylabel('Fitness mean')
plt.xlabel('Number of generations')
plt.title('Population fitness mean and standard deviation')
plt.show()

Let's visualize the trend of the best solution of the population at each epoch.

In [14]:
current_best=[]

for el in ga.fitness_hist:
    current_best.append(el[1])

plt.figure(figsize=(10,10))
plt.plot(current_best)
plt.title('Best fitness of each generation')
plt.xlabel('Number of generations')
plt.ylabel('Fitness')
plt.show()

We observe that the best fitness of each generation presents a jump after around 100 generations after which it remains stable.

Let's visualize the performance of the best solution of the last epoch.

In [15]:
save_path

'/kaggle/working/'

In [16]:
!mkdir 'videos'

mkdir: cannot create directory 'videos': File exists


In [17]:
#Current best solution
idx=np.argmax(ga.fitness)
current_best=ga.population[idx]
model.set_weights(reshape_weights(model,current_best))
create_policy_eval_video(model, env, save_path+'/videos/'+str(ga.num_generations), num_episodes=5)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step


[swscaler @ 0x6247a00] Warning: data is not aligned! This can lead to a speed loss


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━

In [18]:
reward, frames=evaluate_policy(model, env, num_episodes=20)
reward_mean=np.mean(reward)
frames_mean=np.mean(frames)
print('Average reward in 100 episodes {}, average number of frames per episode {}'.format(reward_mean, frames_mean))

(array([ 0.00650635,  1.4197721 ,  0.65901107,  0.39341146, -0.00753247,
       -0.1492759 ,  0.        ,  0.        ], dtype=float32), {})
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━

The agent is able to obtain 200+ cumulative reward averaged over 100 episodes, indicating it can handle successfully a variety of situations.

Let's visualize the performance of the best solution found so far.

In [19]:
model.set_weights(reshape_weights(model, ga.best_solution))
create_policy_eval_video(model, env, 'Best_recorded', num_episodes=5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


[swscaler @ 0x633da00] Warning: data is not aligned! This can lead to a speed loss


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━

Let's compare with the solution that has scored the maximum fitness recorded. Notice that, due to the stocasticity of the environment, this will be biased as a high fitness score may be obtained randomly from some favourable initial conditions.

In [20]:
print('Best fitness:', ga.best_fitness)
print('Best fitness found at generation', np.argmax(ga.fitness_hist))

Best fitness: 224.9833171521093
Best fitness found at generation 167


In [ ]:
reward, frames=evaluate_policy(model, env, num_episodes=100)
reward_mean=np.mean(reward)
frames_mean=np.mean(frames)
print('Average reward in 100 episodes {}, average number of frames per episode {}'.format(reward_mean, frames_mean))

Average reward in 100 episodes 160.76838611153005, average number of frames per episode 758.16


As expected, the average performance over 100 episodes is significatively worse than the performance of the best solution of the last generation.